# 在 AlphaMed 环境中使用异构联邦算法训练自己的模型

## 异构联邦算法调度器简介

当前共有 2 个 HeteroNN 家族的算法调度器：
- `HeteroNNHostScheduler` 和 `HeteroNNCollaboratorScheduler` 对应特征融合时不使用同态加密算法的版本，速度快，但存在协作方特征张量数值泄露给聚合方的风险。
- `SecureHeteroNNHostScheduler` 和 `SecureHeteroNNCollaboratorScheduler` 对应特征融合时使用同态加密算法的版本，能够保证协作方特征张量数值的隐私性，但是训练速度慢很多，且非常消耗内存和网络带宽，因此 batch_size 需要设置的比较小。

模型使用[第 2 节](2.%20%E5%9C%A8%20AlphaMed%20%E5%B9%B3%E5%8F%B0%E4%B8%8A%E8%BF%90%E8%A1%8C%E6%99%AE%E9%80%9A%E6%A8%A1%E5%9E%8B.ipynb)中定义的 `ConvNet` 网络，稍作修改为 `HalfConvNet` 以适应异构联邦示例。

先展示一个使用非同态加密版本训练的示例，然后在此基础上修改为同态加密的版本。

### 异构联邦示例说明

异构联邦的流程相对复杂一些，理解起来需要消耗一些心力，因此先简单介绍一下 HeteroNN 算法的原理和示例的设计思路。

异构联邦场景因为各方原始数据不同，因此各方的抽取的特征也各不相同，需要通过一定的设计来融合各方特征。HeteroNN 的思路是事先定义好各方特征张量的形状，各方据此自行提取自己数据的特征张量，聚合方收集到所有参与方的特征张量后，可以通过拼接、池化、神经网络等各种方式将特征融合，然后送入推理模型得到最终结果。

HeteroNN 只能使用所有参与方共有的样本（各方持有共同样本的不同局部特征数据）作为训练、测试数据，如果一个样本只存在于部分参与方的数据集中，则不能使用这个样本做训练，因为残缺不全的特征会影响模型的运算结果。基于同样的原因，如果一个样本在部分参与方位于训练集中，但在其它参与方位于测试集中，则这个样本既不能用于训练，也不能用于测试。受此影响，使用 HeteroNN 训练时不采用事先划分训练集、测试集的方式。由于隐私求交之前，任何参与方都不知道其它参与方的数据，难以保证所有参与方都能将一个样本统一的划分到训练集或者测试集。HeteroNN 的处理方式是先对各方所有数据执行隐私求交，从而得到各方共用的数据集合。然后各方在此集合上执行相同的数据分割逻辑，从而得到完全相同的训练集和测试集划分。

HeteroNN 支持一主多从的模式，但作为入门示例，此处仅使用一主一从的形式。双方都使用 MNIST 数据集，但其中一方读取数据时将图片左半部分擦除，另一方读取数据时将图片右半部分擦除。双方均只有半张图片，因此不能独立完成识别训练，但是通过 HeteroNN 特征融合后可以恢复出完整的图片数据，进而完成识别训练。

下面分别介绍聚合方和协作方如何配合完成一次异构联邦学习任务，先从聚合方开始。

### `HeteroNNHostScheduler` 介绍

`HeteroNNHostScheduler` 是聚合方的调度器，需要完成两个核心任务：
1. 处理本地训练数据，转化为特征张量，供后续特征融合使用；此处使用的局部模型称为**特征模型**。
2. 收集所有参与方的特征张量，融合后得到样本的完整特征，然后送入业务模型处理；此处使用的局部模型称为**推理模型**。

先介绍一下 `HeteroNNHostScheduler` 定义的接口。

In [11]:
from abc import abstractmethod
from typing import Dict, List, Set, Tuple

import torch
from torch import nn, optim

In [2]:
# 隐私求交相关接口

@abstractmethod
def load_local_ids(self) -> List[str]:
    """返回本地所有数据的 ID 集合。

    必须保证同一个样本在不同的参与方拥有相同的 ID，比如可以使用用户身份证、手机号作为 ID，
    但不能使用各自业务赋予的 UserID。

    此接口必须实现。
    """

@abstractmethod
def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
    """将隐私求交后得到的共有数据 ID 集合划分为训练集和测试集。

    此接口必须实现，且各个参与方的逻辑要相同，保证切分结果在所有参与方一致。

    参数说明:
        id_intersection:
            隐私求交后得到的样本 ID 交集。

    此接口必须实现。
    """

In [7]:
# 模型相关接口

@abstractmethod
def build_feature_model(self) -> nn.Module:
    """返回特征模型的实例。

    此接口必须实现。
    """

@abstractmethod
def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
    """返回特征模型的优化器实例。

    参数说明:
        feature_model:
            特征模型对象，由框架传入。

    此接口必须实现。
    """

@abstractmethod
def build_infer_model(self) -> nn.Module:
    """返回推理模型的实例。

    此接口必须实现。
    """

@abstractmethod
def build_infer_optimizer(self, infer_model: nn.Module) -> optim.Optimizer:
    """返回推理模型的优化器实例。

    参数说明:
        infer_model:
            推理模型对象，由框架传入。

    此接口必须实现。
    """

In [12]:
# 训练相关接口

@abstractmethod
def iterate_train_feature(self,
                          feature_model: nn.Module,
                          train_ids: List[str]) -> Tuple[torch.Tensor, torch.Tensor]:
    """训练数据特征的迭代器，每次返回一个 batch 数据的特征张量和对应标签。

    参数说明:
        feature_model:
            特征模型对象，由框架传入。
        train_ids:
            训练数据样本 ID 列表，由 split_dataset 方法的返回获得。

    此接口必须实现。
    """

@abstractmethod
def iterate_test_feature(self,
                         feature_model: nn.Module,
                         test_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
    """测试数据特征的迭代器，每次返回一个 batch 数据的特征张量和对应标签。

    参数说明:
        feature_model:
            特征模型对象，由框架传入。
        test_ids:
            测试数据样本 ID 列表，由 split_dataset 方法的返回获得。

    此接口必须实现。
    """

@abstractmethod
def train_a_batch(self, feature_projection: Dict[str, torch.Tensor], labels: torch.Tensor):
    """推理模型完成一个 batch 数据的训练，与本地训练模型时的代码相同。

    参数说明:
        feature_projection:
            各个参与方的特征张量映射，key 为各个参与方的 feature_key 设置，后面会有介绍，
            value 为参与方的本地数据特征张量。
        labels:
            训练数据对应标签。

    此接口必须实现。
    """

@abstractmethod
def run_test(self,
             batched_feature_projection: List[Dict[str, torch.Tensor]],
             batched_labels: List[torch.Tensor]):
    """推理模型完成一个 batch 数据的测试，与本地训练模型时的代码相同。

    参数说明:
        batched_feature_projection:
            各个参与方的特征张量映射列表，key 为各个参与方的 feature_key 设置，后面会有介绍，
            value 为参与方的本地数据特征张量。
            在训练阶段，由于每个 batch 的数据要分别计算梯度和更新，所以处理流程是依据批次一批一批
            处理的，每次只返回一个批次的特征张量。在测试阶段，不需要计算梯度和更新，因此将所有批次
            的数据全部处理后，以列表的形式一次性传入 test 方法处理，以大大节省网络和流程开销。
        labels:
            测试数据对应批次标签列表。

    此接口必须实现。
    """

def is_task_finished(self) -> bool:
    """判断训练过程是否结束。

    此接口为可选择实现，默认情况下判断是否达到了设置的 max_round 值，达到了就结束。但如果有特殊需要，
    比如：通过验证集和早停技术避免过拟合时，需要能提前终止训练，则需要重新实现此接口逻辑。
    """

In [13]:
# 其它接口

def validate_context(self):
    """训练开始前验证运行环境，如果发现异常可以提前结束，发送消息通知前台干预。

    此接口为可选择实现，默认只检查模型实例和优化器实例是否成功加载。
    """

接口介绍完毕。

定义算法调度器之前，需要先设计特征模型和推理模型。

特征模型基于前面定义的 `ConvNet` 改造而来。由于本示例中的图片只有一半，与原始图片尺寸不一致，所以需要调整一下网络各层的输入输出维度，改造后的网络称之为 `HalfConvNet`。由于主从双方使用的都是半张图片，所以双方都使用 `HalfConvNet` 作为特征模型。

由于特征和任务都比较简单，推理模型使用两个全连接层加 Softmax 输出分类结果，将推理模型称为 `InferModule`。

In [14]:
from torch import nn
import torch.nn.functional as F


class HalfConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 3))
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=80, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=20)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 80)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        return self.fc2(x)


class InferModule(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(20, 20)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, input):
        out = F.relu(self.fc1(input))
        out = self.fc2(out)
        return F.log_softmax(out, dim=-1)

接下来定义 `DemoHeteroHost(HeteroNNHostScheduler)` 演示如何实现上述接口，完成异构联邦训练的聚合方任务。以下为实现代码和注释：

In [15]:
from hashlib import md5
from time import time
from typing import Dict, List, Set, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from cnn_net import HalfConvNet
from torch.utils.data import DataLoader, Subset

from alphafed import logger
from alphafed.hetero_nn import HeteroNNHostScheduler


# 本示例中使用的推理模型很简单，所以直接在脚本中定义。现实中如果模型比较复杂，
# 也可以参考 HalfConvNet 放在独立文件中然后引入。
class InferModule(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(20, 20)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, input):
        out = F.relu(self.fc1(input))
        out = self.fc2(out)
        return F.log_softmax(out, dim=-1)


class DemoHeteroHost(HeteroNNHostScheduler):

    def __init__(self,
                 feature_key: str,
                 batch_size: int,
                 max_rounds: int = 0,
                 calculation_timeout: int = 300,
                 log_rounds: int = 0) -> None:
        """初始化参数说明.

        以下为 `HeteroNNHostScheduler` 父类定义的初始化参数。
        feature_key:
            当前参与方的特征 key 标识，用于在特征聚合时区分特征来源，以支持区别化处理。
            train_a_batch 和 test 方法中使用的 feature_key 参数既来自于这里。
        max_rounds:
            训练多少轮。
        calculation_timeout:
            本地训练超时时间。
        log_rounds:
            每隔几轮训练执行一次测试，评估记录当前训练效果。

        以下 `DemoHeteroHost` 自定义的扩展初始化参数。
        batch_size:
            训练参数。
        """
        super().__init__(feature_key=feature_key,
                         max_rounds=max_rounds,
                         calculation_timeout=calculation_timeout,
                         log_rounds=log_rounds)
        self.batch_size = batch_size

    def load_local_ids(self) -> List[str]:
        # 聚合方取 MNIST 训练样本的前 20000 个、测试样本的前 5000 个，
        # 协作方取 MNIST 训练样本的第 10000 - 30000 个、测试样本的第 3000 - 7000 个.
        # 求交后共有训练样本 10000 个，测试样本 2000 个。
        # 为避免 ID 冲突，测试样本 ID 全部加 100000，以使其和训练样本 ID 不重合
        train_ids = [str(i) for i in range(0, 20000)]
        test_ids = [str(i) for i in range(100000, 105000)]
        return train_ids + test_ids

    def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
        # 示例中训练样本 ID < 100000，测试样本 ID >= 100000，排序后前 10000 个样本为训练样本 ID。
        ids = [int(_id) for _id in id_intersection]
        ids.sort()
        train_ids = ids[:10000]
        test_ids = ids[10000:]

        logger.info(f'Got {len(train_ids)} intersecting samples for training.')
        logger.info(f'Got {len(test_ids)} intersecting samples for testing.')

        return set(train_ids), set(test_ids)

    def build_feature_model(self) -> nn.Module:
        return HalfConvNet()

    def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
        return optim.SGD(feature_model.parameters(), lr=0.01, momentum=0.9)

    def _erase_right(self, _image: torch.Tensor) -> torch.Tensor:
        return _image[:, :, :, :14]

    def iterate_train_feature(self,
                              feature_model: nn.Module,
                              train_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
        train_dataset = torchvision.datasets.MNIST(
            'data',
            train=True,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        # 每一轮训练时，根据“样本 ID + 当前轮次”的哈希值对样本排序，既可以对每个训练轮次的
        # 样本随机重排序，又可以保证不同参与方随机重排后的样本依然是对齐的。
        train_ids: List = list(train_ids)
        train_ids.sort(key=lambda x: md5(bytes(x + self.current_round)).digest())

        train_dataset = Subset(train_dataset, train_ids)
        self.train_loader = DataLoader(train_dataset,
                                       batch_size=self.batch_size,
                                       shuffle=False)

        for _data, _labels in self.train_loader:
            _data = self._erase_right(_data)
            yield feature_model(_data), _labels

    def iterate_test_feature(self,
                             feature_model: nn.Module,
                             test_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
        test_dataset = torchvision.datasets.MNIST(
            'data',
            train=False,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        test_ids = [(i - 100000) for i in test_ids]
        test_dataset = Subset(test_dataset, test_ids)
        self.test_loader = DataLoader(test_dataset,
                                      batch_size=self.batch_size,
                                      shuffle=False)

        for _data, _labels in self.test_loader:
            _data = self._erase_right(_data)
            yield feature_model(_data), _labels

    def build_infer_model(self) -> nn.Module:
        return InferModule()

    def build_infer_optimizer(self, infer_model: nn.Module) -> optim.Optimizer:
        return optim.SGD(infer_model.parameters(), lr=0.01, momentum=0.9)

    def train_a_batch(self, feature_projection: Dict[str, torch.Tensor], labels: torch.Tensor):
        fusion_tensor = torch.concat((feature_projection['demo_host'],
                                      feature_projection['demo_collaborator']), dim=1)
        self.optimizer.zero_grad()
        out = self.infer_model(fusion_tensor)
        loss = F.nll_loss(out, labels)
        loss.backward()
        self.optimizer.step()

    def run_test(self,
                 batched_feature_projections: List[torch.Tensor],
                 batched_labels: List[torch.Tensor]):
        # 测试时也需要各个参与方提供本地测试数据对应的特征张量
        start = time()
        test_loss = 0
        correct = 0
        for _feature_projection, _lables in zip(batched_feature_projections, batched_labels):
            fusion_tensor = torch.concat((_feature_projection['demo_host'],
                                          _feature_projection['demo_collaborator']), dim=1)
            out: torch.Tensor = self.infer_model(fusion_tensor)
            test_loss += F.nll_loss(out, _lables)
            pred = out.max(1, keepdim=True)[1]
            correct += pred.eq(_lables.view_as(pred)).sum().item()

        end = time()

        test_loss /= len(self.test_loader.dataset)
        accuracy = correct / len(self.test_loader.dataset)
        correct_rate = 100. * accuracy

        self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
        self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
        self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
        self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

聚合方特征模型、推理模型、以及训练的调度器 `DemoHeteroHost` 设计完毕。接下来介绍协作方需要完成的工作。

### `HeteroNNCollaboratorScheduler` 介绍

`HeteroNNCollaboratorScheduler` 是协作方的调度器。协作方的任务比较简单，将本地训练、测试数据转化为特征张量发给聚合方即可。模型训练时聚合方处理完特征数据后，会回传梯度供协作方更新本地特征模型。

先介绍一下 `HeteroNNCollaboratorScheduler` 定义的接口，注意协作方的接口是聚合方的子集。

In [17]:
# 隐私求交相关接口

@abstractmethod
def load_local_ids(self) -> List[str]:
    """返回本地所有数据的 ID 集合。

    必须保证同一个样本在不同的参与方拥有相同的 ID，比如可以使用用户身份证、手机号作为 ID，
    但不能使用各自业务赋予的 UserID。

    此接口必须实现。
    """

@abstractmethod
def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
    """将隐私求交后得到的共有数据 ID 集合划分为训练集和测试集。

    此接口必须实现，且各个参与方的逻辑要相同，保证切分结果在所有参与方一致。

    参数说明:
        id_intersection:
            隐私求交后得到的样本 ID 交集。

    此接口必须实现。
    """

In [18]:
# 模型相关接口，无须再定义推理模型

@abstractmethod
def build_feature_model(self) -> nn.Module:
    """返回特征模型的实例。

    此接口必须实现。
    """

@abstractmethod
def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
    """返回特征模型的优化器实例。

    参数说明:
        feature_model:
            特征模型对象，由框架传入。

    此接口必须实现。
    """

In [19]:
# 训练相关接口，无须定义训练、测试逻辑

@abstractmethod
def iterate_train_feature(self,
                          feature_model: nn.Module,
                          train_ids: List[str]) -> torch.Tensor:
    """训练数据特征的迭代器，每次返回一个 batch 数据的特征张量。

    注意这里的返回值与聚合方不同，聚合方需要返回特征张量和标签，参与方只需要返回特征张量。

    参数说明:
        feature_model:
            特征模型对象，由框架传入。
        train_ids:
            训练数据样本 ID 列表，由 split_dataset 方法的返回获得。

    此接口必须实现。
    """

@abstractmethod
def iterate_test_feature(self,
                         feature_model: nn.Module,
                         test_ids: Set[str]) -> torch.Tensor:
    """测试数据特征的迭代器，每次返回一个 batch 数据的特征张量。

    注意这里的返回值与聚合方不同，聚合方需要返回特征张量和标签，参与方只需要返回特征张量。

    参数说明:
        feature_model:
            特征模型对象，由框架传入。
        test_ids:
            测试数据样本 ID 列表，由 split_dataset 方法的返回获得。

    此接口必须实现。
    """

In [20]:
# 其它接口

def validate_context(self):
    """训练开始前验证运行环境，如果发现异常可以提前结束，发送消息通知前台干预。

    此接口为可选择实现，默认只检查模型实例和优化器实例是否成功加载。
    """

接口介绍完毕，可见所有接口都在介绍聚合方调度器 `HeteroNNHostScheduler` 时出现过，是其接口的子集。

在本示例中，由于输入数据类型一致，参与方使用和聚合方相同的网络作为特征模型。如果双方数据不一致，调整为各自定义即可，不会影响学习流程。同理二者使用的模型优化器也是独立的，互不影响。

接下来定义 `DemoHeteroCollaborator(HeteroNNCollaboratorScheduler)` 演示如何实现上述接口，完成异构联邦训练的协作方任务。以下为实现代码和注释：

In [21]:
from hashlib import md5
from typing import List, Set, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from cnn_net import HalfConvNet
from torch.utils.data import DataLoader, Subset

from alphafed import logger
from alphafed.hetero_nn import HeteroNNCollaboratorScheduler


class DemoHeteroCollaborator(HeteroNNCollaboratorScheduler):

    def __init__(self,
                 feature_key: str,
                 batch_size: int,
                 schedule_timeout: int = 30,
                 is_feature_trainable: bool = True) -> None:
        super().__init__(feature_key=feature_key,
                         schedule_timeout=schedule_timeout,
                         is_feature_trainable=is_feature_trainable)
        self.batch_size = batch_size

    def load_local_ids(self) -> List[str]:
        # 聚合方取 MNIST 训练样本的前 20000 个、测试样本的前 5000 个，
        # 协作方取 MNIST 训练样本的第 10000 - 30000 个、测试样本的第 3000 - 7000 个.
        # 求交后共有训练样本 10000 个，测试样本 2000 个。
        # 为避免 ID 冲突，测试样本 ID 全部加 100000，以使其和训练样本 ID 不重合
        train_ids = [str(i) for i in range(10000, 30000)]
        test_ids = [str(i) for i in range(103000, 107000)]
        return train_ids + test_ids

    def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
        # 示例中训练样本 ID < 100000，测试样本 ID >= 100000，排序后前 10000 个样本为训练样本 ID。
        ids = [int(_id) for _id in id_intersection]
        ids.sort()
        train_ids = ids[:10000]
        test_ids = ids[10000:]

        logger.info(f'Got {len(train_ids)} intersecting samples for training.')
        logger.info(f'Got {len(test_ids)} intersecting samples for testing.')

        return set(train_ids), set(test_ids)

    def build_feature_model(self) -> nn.Module:
        # 现实中每个协作方都可以定义自己的特征模型，不需要与其它任何参与方相同
        return HalfConvNet()

    def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
        # 现实中每个协作方都可以定义自己的优化器，不需要与其它任何参与方相同
        return optim.SGD(feature_model.parameters(), lr=0.01, momentum=0.9)

    def _erase_left(self, _image: torch.Tensor) -> torch.Tensor:
        return _image[:, :, :, 14:]

    def iterate_train_feature(self,
                              feature_model: nn.Module,
                              train_ids: Set[str]) -> torch.Tensor:
        train_dataset = torchvision.datasets.MNIST(
            'data',
            train=True,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        train_ids: List = list(train_ids)
        train_ids.sort(key=lambda x: md5(bytes(x + self.current_round)).digest())

        train_dataset = Subset(train_dataset, train_ids)
        self.train_loader = DataLoader(train_dataset,
                                       batch_size=self.batch_size,
                                       shuffle=False)

        for _data, _ in self.train_loader:  # 丢弃标签数据
            _data = self._erase_left(_data)
            yield feature_model(_data)

    def iterate_test_feature(self,
                             feature_model: nn.Module,
                             test_ids: Set[str]) -> torch.Tensor:
        test_dataset = torchvision.datasets.MNIST(
            'data',
            train=False,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        test_ids = [(i - 100000) for i in test_ids]
        test_dataset = Subset(test_dataset, test_ids)
        self.test_loader = DataLoader(test_dataset,
                                      batch_size=self.batch_size,
                                      shuffle=False)

        for _data, _ in self.test_loader:  # 丢弃标签数据
            _data = self._erase_left(_data)
            yield feature_model(_data)

协作方特征模型以及训练的调度器 `DemoHeteroCollaborator` 设计完毕。接下来演示如何调试运行，之后再介绍使用同态加密版本的 SecureHeteroNN 调度器。

### 使用 `HeteroNN` 调度器模拟训练模型

先介绍模拟调试环节，假设此时还不确定 `DemoHeteroHost` 的代码是否正确，能不能跑起来。可以这样做调试：
1. 将 `DemoHeteroHost` 的代码复制粘贴到 Notebook 的一个 Cell 单元中，运行一次完成加载。加载时如果存在错误，Notebook 会显示异常信息。（可以故意修改代码制造语法错误，模拟体验这种情况。）
2. 加载成功之后需要在模拟环境中模拟运行一下试试。如果 `DemoHeteroCollaborator` 的代码是协作方编写的，且不会提供给聚合方，则无法在本地模拟环境完成联调，只能等正式运行之后根据运行情况解决问题。这个限制对于协作方来讲也是一样的。
3. 在我们的示例中，假设可以得到协作方的代码及少数样例数据，则可以分别打开两个 Notebook 脚本文件，分别模拟聚合方与协作方，加载 `DemoHeteroHost` 和 `DemoHeteroCollaborator`。参考下面的代码分别实例化聚合方协作方，通过 `with mock_context():` 在模拟环境中调用开始计算的接口。（实际运行时任务管理器会自动完成实例化和调用。）运行中产生的文件数据默认会保存在“{NODE_ID}/{TASK_ID}”目录下，其中 NODE_ID 是测试脚本启动模拟环境时设置的 `id` 值，TASK_ID 是测试脚本启动调度器时传入的 `task_id` 值。

> 1. 有关模拟环境的详细信息将在后续章节补充介绍，当前不需要深究，不会影响对算法调度器的理解。
> 2. 不能在一个 Notebook 脚本文件中模拟多个节点是因为受到技术上的制约，Notebook 环境不支持一个 Notebook 脚本文件中的多个 cell 并行，只能挨个串行。而联邦学习是并行运算，所以不能使用多个 cell 模拟多个节点。

In [ ]:
from alphafed import mock_context


# 聚合方的模拟启动脚本
scheduler = DemoHeteroHost(
    feature_key='demo_host',
    batch_size=128,  # 各个参与方需要设置相同的 batch_size，否则数据无法对齐
    max_rounds=5,  # 受本地资源限制，运行速度可能会很慢，调试时不建议设置太高
    calculation_timeout=1800,  # 受本地资源限制，运行速度可能会很慢，设置太低容易导致超时
    log_rounds=1
)

aggregator_id = '9afb99da-2e4c-4676-a434-5f312f583947'  # 设置一个假想 ID
task_id = '5a8c6b50-6267-4258-afb0-a4e7413ead58'  # 设置一个假想 ID
# 算法实际运行时会从任务管理器获取任务参与节点的 Node ID 列表，但是在模拟环境不能通过
# 访问实际接口获得这个信息，所以需要通过 nodes 参数将这个列表配置在模拟环境中。
collaborate_id = 'ec72a25c-ff83-47f2-a4bc-74321acace99'  # 设置一个假想 ID
with mock_context(id=aggregator_id, nodes=[aggregator_id, collaborate_id]):  # 在模拟调试环境中运行
    # _run 接口是最底层的 Scheduler 接口定义的，横向联邦、异构联邦框架都实现了对应接口
    scheduler._run(id=aggregator_id, task_id=task_id, is_initiator=True)


# 参与方的模拟启动脚本，需要复制到另一个 Notebook 脚本文件中执行
# 与横向联邦不同，异构联邦中 scheduler 实例和聚合方的不一样
scheduler = DemoHeteroCollaborator(
    feature_key='demo_collaborator',
    batch_size=128  # 各个参与方需要设置相同的 batch_size，否则数据无法对齐
)
collaborate_id = 'ec72a25c-ff83-47f2-a4bc-74321acace99'  # 需要与聚合方设置的 ID 一致
task_id = '5a8c6b50-6267-4258-afb0-a4e7413ead58'  # 需要与聚合方设置的 ID 一致
aggregator_id = '9afb99da-2e4c-4676-a434-5f312f583947'  # 需要与聚合方设置的 ID 一致
# 算法实际运行时会从任务管理器获取任务参与节点的 Node ID 列表，但是在模拟环境不能通过
# 访问实际接口获得这个信息，所以需要通过 nodes 参数将这个列表配置在模拟环境中。
with mock_context(id=collaborate_id, nodes=[aggregator_id, collaborate_id]):  # 在模拟调试环境中运行
    # _run 接口是最底层的 Scheduler 接口定义的，横向联邦、异构联邦框架都实现了对应接口
    scheduler._run(id=collaborate_id, task_id=task_id)

整理好的[聚合方脚本](res/4_aggregator.ipynb)、[协作方脚本](res/4_collaborator.ipynb)均可直接运行。

在模拟环境调试运行成功之后，对聚合方、参与方的启动脚本稍作修改，就可以启动正式任务了。下面是修改后的正式启动任务的脚步代码及注释。执行异构联邦学习任务需要登录 [AlphaMed Playground 客户端](http://playground.ssplabs.com/)，[这里](../hetero_nn/README.md)有创建异构联邦学习任务的详细说明，请按照说明中的步骤运行示例程序。注意异构任务需要各个参与方在自己的 Playground 中各自启动任务，而不像横向联邦任务中只需要聚合方一方启动任务。

In [ ]:
# 聚合方的任务启动脚本
scheduler = DemoHeteroHost(
    feature_key='demo_host',
    batch_size=128,  # 各个参与方需要设置相同的 batch_size，否则数据无法对齐
    max_rounds=5,  # 受本地资源限制，运行速度可能会很慢，调试时不建议设置太高
    calculation_timeout=1800,  # 受本地资源限制，运行速度可能会很慢，设置太低容易导致超时
    log_rounds=1
)

# 这些模拟调试的代码不需要了
# aggregator_id = '9afb99da-2e4c-4676-a434-5f312f583947'  # 随便设置一个
# task_id = '5a8c6b50-6267-4258-afb0-a4e7413ead58'  # 随便设置一个
# # 算法实际运行时会从任务管理器获取任务参与节点的 Node ID 列表，但是在模拟环境不能通过
# # 访问实际接口获得这个信息，所以需要通过 nodes 参数将这个列表配置在模拟环境中。
# nodes = [
#     '9afb99da-2e4c-4676-a434-5f312f583947',
#     'ec72a25c-ff83-47f2-a4bc-74321acace99'
# ]
# with mock_context(id=aggregator_id, nodes=nodes):  # 在模拟调试环境中运行
#     # _run 接口是最底层的 Scheduler 接口定义的，横向联邦、异构联邦都一样
#     scheduler._run(id=aggregator_id, task_id=task_id, is_initiator=True)

scheduler.launch_task(task_id='YOUR_TASK_ID')


# 参与方的任务启动脚本
scheduler = DemoHeteroCollaborator(feature_key='demo_collaborator', batch_size=128)

# 这些模拟调试的代码不需要了
# collaborate_id = 'ec72a25c-ff83-47f2-a4bc-74321acace99'  # 随便设置一个
# task_id = '5a8c6b50-6267-4258-afb0-a4e7413ead58'  # 与聚合方一致
# # 算法实际运行时会从任务管理器获取任务参与节点的 Node ID 列表，但是在模拟环境不能通过
# # 访问实际接口获得这个信息，所以需要通过 nodes 参数将这个列表配置在模拟环境中。
# nodes = [
#     '9afb99da-2e4c-4676-a434-5f312f583947',
#     'ec72a25c-ff83-47f2-a4bc-74321acace99'
# ]
# with mock_context(id=collaborate_id, nodes=nodes):  # 在模拟调试环境中运行
#     # _run 接口是最底层的 Scheduler 接口定义的，横向联邦、异构联邦都一样
#     scheduler._run(id=collaborate_id, task_id=task_id)

scheduler.launch_task(task_id='YOUR_TASK_ID')

### `SecureHeteroNNHostScheduler` 介绍

`SecureHeteroNNHostScheduler` 调度器在聚合方、协作方之间传输特征、梯度张量数据时，会使用同态加密的方式保护数据原始值，从而保证训练、推理过程中不会出现任何数据泄露，以适应安全要求更高的场合。

前面使用 `HeteroNNHostScheduler` 调度器时，协作方生成的特征张量是以原始值的形式发送给聚合方的，所以聚合方在做特征融合时可以随意选择融合方式。但是 `SecureHeteroNNHostScheduler` 调度器由于使用了同态加密机制，而同态加密算法只支持线性运算，因此特征融合时不允许使用非线性操作。同态加密处理特征融合和梯度更新的操作很复杂，幸运的是 AlphaMed 平台已经提供了一套实现，消除了开发者自行处理同态加密数据的麻烦。开发者只需要提供一个简单的配置，告诉平台数据融合时每个参与方原始特征向量的维度和线性变换后的特征向量的维度，其余的事情都可以交给平台处理。以下是一个配置的示例：

In [23]:
project_layer_config = [
    ('demo_host', 10, 10),
    ('demo_collaborator', 10, 10)
]

其中的 'demo_host'，'demo_collaborator', 对应各个参与方的 `feature_key` 参数。后面的两个整数依次代表本地样本特征向量的原始维度、和经过线性变换后的维度。所有参与方特征线性变换后的维度之和，需要匹配推理模型的输入维度。除此之外 `SecureHeteroNNHostScheduler` 的实现与非加密版本的 `HeteroNNHostScheduler` 完全一致。

接下来定义 `DemoSecureHeteroHost(SecureHeteroNNHostScheduler)` 演示如何使用同态加密的调度器，完成异构联邦训练的聚合方任务。实际上，与前面定义的 `DemoHeteroHost` 相比，`DemoSecureHeteroHost` 唯一的不同就是初始化时需要提供两个新的特征融合超参数 `project_layer_config` 和 `project_layer_lr`，其它方面完全一致。

In [24]:
from hashlib import md5
from time import time
from typing import Dict, List, Set, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from cnn_net import HalfConvNet
from torch.utils.data import DataLoader, Subset

from alphafed import logger
from alphafed.hetero_nn import SecureHeteroNNHostScheduler


class InferModule(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(20, 20)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, input):
        out = F.relu(self.fc1(input))
        out = self.fc2(out)
        return F.log_softmax(out, dim=-1)


class DemoSecureHeteroHost(SecureHeteroNNHostScheduler):

    def __init__(self,
                 feature_key: str,
                 project_layer_config: List[Tuple[str, int, int]],
                 project_layer_lr: float,
                 batch_size: int,
                 max_rounds: int = 0,
                 calculation_timeout: int = 300,
                 log_rounds: int = 0) -> None:
        """初始化参数说明.

        以下为 `HeteroNNHostScheduler` 父类定义的初始化参数。
        feature_key:
            当前参与方的特征 key 标识，用于在特征聚合时区分特征来源，以支持区别化处理。
            train_a_batch 和 test 方法中使用的 feature_key 参数既来自于这里。
        project_layer_config:
            特征融合变换配置，其中每一个配置项是一个三元组，分别设置：
            feature_key，特征向量原始维度，特征向量线性变换输出维度。
        project_layer_lr:
            特征融合参数的学习率。
        max_rounds:
            训练多少轮。
        calculation_timeout:
            本地训练超时时间。
        log_rounds:
            每隔几轮训练执行一次测试，评估记录当前训练效果。

        以下 `DemoHeteroHost` 自定义的扩展初始化参数。
        batch_size:
            训练参数。
        """
        super().__init__(feature_key=feature_key,
                         project_layer_config=project_layer_config,
                         project_layer_lr=project_layer_lr,
                         max_rounds=max_rounds,
                         calculation_timeout=calculation_timeout,
                         log_rounds=log_rounds)
        self.batch_size = batch_size

    def load_local_ids(self) -> List[str]:
        # 聚合方取 MNIST 训练样本的前 20000 个、测试样本的前 5000 个，
        # 协作方取 MNIST 训练样本的第 10000 - 30000 个、测试样本的第 3000 - 7000 个.
        # 求交后共有训练样本 10000 个，测试样本 2000 个。
        # 为避免 ID 冲突，测试样本 ID 全部加 100000，以使其和训练样本 ID 不重合
        train_ids = [str(i) for i in range(0, 20000)]
        test_ids = [str(i) for i in range(100000, 105000)]
        return train_ids + test_ids

    def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
        # 示例中训练样本 ID < 100000，测试样本 ID >= 100000，排序后前 10000 个样本为训练样本 ID。
        ids = [int(_id) for _id in id_intersection]
        ids.sort()
        train_ids = ids[:10000]
        test_ids = ids[10000:]

        logger.info(f'Got {len(train_ids)} intersecting samples for training.')
        logger.info(f'Got {len(test_ids)} intersecting samples for testing.')

        return set(train_ids), set(test_ids)

    def build_feature_model(self) -> nn.Module:
        return HalfConvNet()

    def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
        return optim.SGD(feature_model.parameters(), lr=0.01, momentum=0.9)

    def _erase_right(self, _image: torch.Tensor) -> torch.Tensor:
        return _image[:, :, :, :14]

    def iterate_train_feature(self,
                              feature_model: nn.Module,
                              train_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
        train_dataset = torchvision.datasets.MNIST(
            'data',
            train=True,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        # 每一轮训练时，根据“样本 ID + 当前轮次”的哈希值对样本排序，既可以对每个训练轮次的
        # 样本随机重排序，又可以保证不同参与方随机重排后的样本依然是对齐的。
        train_ids: List = list(train_ids)
        train_ids.sort(key=lambda x: md5(bytes(x + self.current_round)).digest())

        train_dataset = Subset(train_dataset, train_ids)
        self.train_loader = DataLoader(train_dataset,
                                       batch_size=self.batch_size,
                                       shuffle=False)

        for _data, _labels in self.train_loader:
            _data = self._erase_right(_data)
            yield feature_model(_data), _labels

    def iterate_test_feature(self,
                             feature_model: nn.Module,
                             test_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
        test_dataset = torchvision.datasets.MNIST(
            'data',
            train=False,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        test_ids = [(i - 100000) for i in test_ids]
        test_dataset = Subset(test_dataset, test_ids)
        self.test_loader = DataLoader(test_dataset,
                                      batch_size=self.batch_size,
                                      shuffle=False)

        for _data, _labels in self.test_loader:
            _data = self._erase_right(_data)
            yield feature_model(_data), _labels

    def build_infer_model(self) -> nn.Module:
        return InferModule()

    def build_infer_optimizer(self, infer_model: nn.Module) -> optim.Optimizer:
        return optim.SGD(infer_model.parameters(), lr=0.01, momentum=0.9)

    def train_a_batch(self, feature_projection: Dict[str, torch.Tensor], labels: torch.Tensor):
        fusion_tensor = torch.concat((feature_projection['demo_host'],
                                      feature_projection['demo_collaborator']), dim=1)
        self.optimizer.zero_grad()
        out = self.infer_model(fusion_tensor)
        loss = F.nll_loss(out, labels)
        loss.backward()
        self.optimizer.step()

    def run_test(self,
                 batched_feature_projections: List[torch.Tensor],
                 batched_labels: List[torch.Tensor]):
        # 测试时也需要各个参与方提供本地测试数据对应的特征张量
        start = time()
        test_loss = 0
        correct = 0
        for _feature_projection, _lables in zip(batched_feature_projections, batched_labels):
            fusion_tensor = torch.concat((_feature_projection['demo_host'],
                                          _feature_projection['demo_collaborator']), dim=1)
            out: torch.Tensor = self.infer_model(fusion_tensor)
            test_loss += F.nll_loss(out, _lables)
            pred = out.max(1, keepdim=True)[1]
            correct += pred.eq(_lables.view_as(pred)).sum().item()

        end = time()

        test_loss /= len(self.test_loader.dataset)
        accuracy = correct / len(self.test_loader.dataset)
        correct_rate = 100. * accuracy

        self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
        self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
        self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
        self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

聚合方特征模型、推理模型、以及训练的调度器 `DemoSecureHeteroHost` 设计完毕。接下来介绍协作方需要完成的工作。

### `SecureHeteroNNCollaboratorScheduler` 介绍

协作方不需要处理参数聚合，因此加密版的协作方调度器 `DemoSecureHeteroCollaborator` 与 `DemoHeteroCollaborator` 几乎完全一致，唯一区别的是多了一个参数 `project_layer_lr`。设置这个值不表示协作方也需要参与特征融合层参数更新的工作，而是需要这个值来解密出正确的梯度值，所以需要与聚合方的配置保持一致。接下来定义 `DemoSecureHeteroCollaborator(SecureHeteroNNCollaboratorScheduler)` 演示如何使用同态加密的调度器，完成异构联邦训练的协作方任务。

In [25]:
from hashlib import md5
from typing import List, Set, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from cnn_net import HalfConvNet
from torch.utils.data import DataLoader, Subset

from alphafed import logger
from alphafed.hetero_nn import SecureHeteroNNCollaboratorScheduler


class DemoSecureHeteroCollaborator(SecureHeteroNNCollaboratorScheduler):

    def __init__(self,
                 feature_key: str,
                 project_layer_lr: int,
                 batch_size: int,
                 schedule_timeout: int = 30,
                 is_feature_trainable: bool = True) -> None:
        super().__init__(feature_key=feature_key,
                         project_layer_lr=project_layer_lr,
                         schedule_timeout=schedule_timeout,
                         is_feature_trainable=is_feature_trainable)
        self.batch_size = batch_size

    def load_local_ids(self) -> List[str]:
        # 聚合方取 MNIST 训练样本的前 20000 个、测试样本的前 5000 个，
        # 协作方取 MNIST 训练样本的第 10000 - 30000 个、测试样本的第 3000 - 7000 个.
        # 求交后共有训练样本 10000 个，测试样本 2000 个。
        # 为避免 ID 冲突，测试样本 ID 全部加 100000，以使其和训练样本 ID 不重合
        train_ids = [str(i) for i in range(10000, 30000)]
        test_ids = [str(i) for i in range(103000, 107000)]
        return train_ids + test_ids

    def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
        # 示例中训练样本 ID < 100000，测试样本 ID >= 100000，排序后前 10000 个样本为训练样本 ID。
        ids = [int(_id) for _id in id_intersection]
        ids.sort()
        train_ids = ids[:10000]
        test_ids = ids[10000:]

        logger.info(f'Got {len(train_ids)} intersecting samples for training.')
        logger.info(f'Got {len(test_ids)} intersecting samples for testing.')

        return set(train_ids), set(test_ids)

    def build_feature_model(self) -> nn.Module:
        # 现实中每个协作方都可以定义自己的特征模型，不需要与其它任何参与方相同
        return HalfConvNet()

    def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
        # 现实中每个协作方都可以定义自己的优化器，不需要与其它任何参与方相同
        return optim.SGD(feature_model.parameters(), lr=0.01, momentum=0.9)

    def _erase_left(self, _image: torch.Tensor) -> torch.Tensor:
        return _image[:, :, :, 14:]

    def iterate_train_feature(self,
                              feature_model: nn.Module,
                              train_ids: Set[str]) -> torch.Tensor:
        train_dataset = torchvision.datasets.MNIST(
            'data',
            train=True,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        train_ids: List = list(train_ids)
        train_ids.sort(key=lambda x: md5(bytes(x + self.current_round)).digest())

        train_dataset = Subset(train_dataset, train_ids)
        self.train_loader = DataLoader(train_dataset,
                                       batch_size=self.batch_size,
                                       shuffle=False)

        for _data, _ in self.train_loader:  # 丢弃标签数据
            _data = self._erase_left(_data)
            yield feature_model(_data)

    def iterate_test_feature(self,
                             feature_model: nn.Module,
                             test_ids: Set[str]) -> torch.Tensor:
        test_dataset = torchvision.datasets.MNIST(
            'data',
            train=False,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        test_ids = [(i - 100000) for i in test_ids]
        test_dataset = Subset(test_dataset, test_ids)
        self.test_loader = DataLoader(test_dataset,
                                      batch_size=self.batch_size,
                                      shuffle=False)

        for _data, _ in self.test_loader:  # 丢弃标签数据
            _data = self._erase_left(_data)
            yield feature_model(_data)

协作方特征模型以及训练的调度器 `DemoSecureHeteroCollaborator` 设计完毕。加密版异构联邦学习任务的调试、运行方式与非加密版完全一致，因此不再赘述，感兴趣的读者可以自行尝试。需要说明的是，同态加密运算对计算资源的要求非常的高，因此对运行环境有较高的要求，且运行时间会比普通版长很多，需要很耐心的等待计算完成。